In [34]:
import math
import os
import subprocess
import sys
from collections import deque

from astropy.io import ascii
from astropy.coordinates import SkyCoord, Angle
from astropy.io import fits, votable
from astropy.wcs import WCS
import numpy as np
import numpy.core.records as rec
from astropy import units as u
from astropy.table import QTable, Table, Column


## Produce source cutout scripts

Produce scripts to extarct cutouts around known sources with the aim of producing absorption specra.

Each script will produce a cubelet from the ASKAP data around a known source excluding the shortest baselinesand thus limiiting the large scale emission present in the data. The scripts use CASA.

In [4]:
# Read and filter catalogue
src_votable = votable.parse('AS037_Continuum_Component_Catalogue_8906_100.votable', pedantic=False)
table = src_votable.get_first_table().to_table()
targets = table[table['flux_peak']>15]
targets

id,catalogue_id,first_sbid,other_sbids,project_id,island_id,component_id,component_name,ra_hms_cont,dec_dms_cont,ra_deg_cont,dec_deg_cont,ra_err,dec_err,freq,flux_peak,flux_peak_err,flux_int,flux_int_err,maj_axis,min_axis,pos_ang,maj_axis_err,min_axis_err,pos_ang_err,maj_axis_deconv,min_axis_deconv,maj_axis_deconv_err,pos_ang_deconv,min_axis_deconv_err,pos_ang_deconv_err,chi_squared_fit,rms_fit_gauss,spectral_index,spectral_index_err,spectral_curvature,spectral_curvature_err,rms_image,has_siblings,fit_is_estimate,spectral_index_from_tt,flag_c4,comment,quality_level,released_date
,,,,,,,,,,deg,deg,arcsec,arcsec,MHz,mJy/beam,mJy/beam,mJy,mJy,arcsec,arcsec,deg,arcsec,arcsec,deg,arcsec,arcsec,arcsec,deg,arcsec,deg,,mJy/beam,,,,,mJy/beam,,,,,,,
int64,int64,int32,bytes500,int64,bytes255,bytes256,bytes32,bytes16,bytes16,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,bytes1000,bytes15,bytes24
624901,100,8906,,15,SB8906_island_9,SB8906_component_9a,J001932-710022,00:19:32.9,-71:00:22,4.887137,-71.006189,0.02,0.01,1420.5,309.726,1.24,323.813,1.853,9.45,7.43,99.03,0.04,0.01,0.67,1.88,1.62,4.99,-69.0,6.84,12.8,8347.932,8340.629,-99.0,0.0,-99.0,0.0,0.323,0,0,--,1,,NOT_VALIDATED,
624900,100,8906,,15,SB8906_island_99,SB8906_component_99a,J005813-700958,00:58:13.2,-70:09:58,14.555193,-70.166284,0.02,0.02,1420.5,43.212,0.226,44.957,0.337,9.69,7.22,109.93,0.05,0.01,0.71,3.48,0.0,0.09,-45.5,0.0,0.87,189.555,1451.265,-99.0,0.0,-99.0,0.0,0.333,0,0,--,0,,NOT_VALIDATED,
624887,100,8906,,15,SB8906_island_98,SB8906_component_98a,J011134-711413,01:11:34.5,-71:14:13,17.893699,-71.237148,0.01,0.01,1420.5,45.35,0.17,46.352,0.251,9.08,7.57,117.73,0.03,0.01,0.84,3.3,0.0,0.04,-15.69,0.0,0.29,206.052,1715.694,-99.0,0.0,-99.0,0.0,0.207,0,0,--,0,,NOT_VALIDATED,
624876,100,8906,,15,SB8906_island_97,SB8906_component_97a,J002248-734007,00:22:48.0,-73:40:07,5.700154,-73.668876,0.01,0.01,1420.5,43.777,0.117,53.542,0.197,10.21,8.05,96.67,0.03,0.01,0.45,4.32,3.45,0.1,86.4,0.17,1.46,63.182,833.254,-99.0,0.0,-99.0,0.0,0.326,0,0,--,0,,NOT_VALIDATED,
624780,100,8906,,15,SB8906_island_91,SB8906_component_91a,J014114-740731,01:41:14.6,-74:07:31,25.310883,-74.125381,0.02,0.02,1420.5,47.153,0.68,53.782,0.854,9.68,7.92,105.28,0.07,0.01,0.75,3.58,2.25,0.2,-23.69,0.57,1.36,323.868,1275.726,-99.0,0.0,-99.0,0.0,0.188,1,0,--,0,,NOT_VALIDATED,
624764,100,8906,,15,SB8906_island_90,SB8906_component_90a,J012430-752241,01:24:30.2,-75:22:41,21.125805,-75.378293,0.02,0.02,1420.5,49.636,0.215,62.406,0.37,9.78,8.64,160.24,0.04,0.01,1.42,6.32,0.0,0.01,-2.02,0.0,0.2,48.0,789.542,-99.0,0.0,-99.0,0.0,0.63,0,0,--,0,,NOT_VALIDATED,
624751,100,8906,,15,SB8906_island_8,SB8906_component_8a,J010218-754651,01:02:18.9,-75:46:51,15.578771,-75.780991,0.01,0.01,1420.5,320.5,0.781,362.115,1.236,9.47,8.02,104.64,0.02,0.01,0.59,3.57,1.66,0.06,-7.11,0.34,0.54,2144.309,3404.535,-99.0,0.0,-99.0,0.0,0.514,1,0,--,1,,NOT_VALIDATED,
624750,100,8906,,15,SB8906_island_89,SB8906_component_89a,J013742-733049,01:37:42.4,-73:30:49,24.426579,-73.513716,0.02,0.01,1420.5,48.008,0.187,50.501,0.281,9.35,7.56,108.85,0.04,0.01,0.74,2.98,0.0,0.12,-23.98,0.0,0.57,252.157,1753.592,-99.0,0.0,-99.0,0.0,0.228,0,0,--,0,,NOT_VALIDATED,


In [5]:
def slice_strings(a,start,end):
    if end is None:
        if start > 0:
            raise('end must be present when start is positive')
        b = a.view((str,1)).reshape(len(a),-1)[:,start:]
        return np.frombuffer(b.tostring(),dtype=(str,start*-1))
    
    b = a.view((str,1)).reshape(len(a),-1)[:,start:end]
    return np.frombuffer(b.tostring(),dtype=(str,end-start))

In [6]:
# Build the list of beam locations
beams = ascii.read('beam_listing_SB8906_casda.csv', format='no_header', guess=False, delimiter=',')
names = ('col1','col2', 'col3', 'col4')
new_names = ('filename','interleave', 'ra_rad', 'dec_rad')
beams.rename_columns(names, new_names)

beam_id = slice_strings(beams['filename'], -8, -6)
interleave_id = slice_strings(beams['interleave'], -1, None)# beams['interleave'][:][-1:]
file_interleave = slice_strings(beams['filename'], -14, -13)
#ids = np.vstack([[beam_id], [interleave_id]])
ids = np.stack((beam_id, interleave_id), axis=-1)
unique_ids, unique_idx = np.unique(ids, axis=0, return_index=True)
#print (ids, unique_ids)
beams['beam_id'] = beam_id
beams['interleave'] = interleave_id
beams['file_interleave'] = file_interleave
unique_beams = beams[beams['interleave'] == beams['file_interleave']]

u_beam_locs = SkyCoord(ra=unique_beams['ra_rad']*u.rad, dec=unique_beams['dec_rad']*u.rad, frame='icrs')
u_beam_locs


<SkyCoord (ICRS): (ra, dec) in deg
    [( 4.61342858, -71.11972947), ( 5.36703937, -71.98813967),
     ( 6.19226404, -72.85311344), ( 7.10039761, -73.71437877),
     ( 8.10514534, -74.57155345), ( 9.22327392, -75.42411422),
     ( 7.33238309, -71.33410344), ( 8.2207335 , -72.19043927),
     ( 9.19429692, -73.0422068 ), (10.26629912, -73.88893516),
     (11.45269709, -74.73000152), (12.77287374, -75.56458939),
     ( 9.16614835, -70.66059468), (10.10510509, -71.50815499),
     (11.1299407 , -72.35033114), (12.2531687 , -73.18658124),
     (13.48968891, -74.01620609), (14.85733382, -74.83830922),
     (11.8576096 , -70.80819561), (12.92183507, -71.64083771),
     (14.08305696, -72.46673777), (15.35500447, -73.28514139),
     (16.75388286, -74.09509871), (18.29888247, -74.89541691),
     (13.4896866 , -70.09223223), (14.58338702, -70.91551115),
     (15.7717441 , -71.73130586), (17.06729156, -72.53882537),
     (18.48463387, -73.33708868), (20.04082028, -74.12488257),
     (16.12681685, -

In [7]:
def get_beams_near_src(target_loc, beam_locs, beams, max_sep = 0.8*(1*u.deg)):
    beam_sep = u_beam_locs.separation(target_loc)
    beams_covering_target = beam_sep < max_sep
    src_beams = unique_beams[beams_covering_target]
    src_beam_sep = beam_sep[beams_covering_target]
    return src_beams, src_beam_sep



In [8]:
comp_names = []
comp_ra = []
comp_dec = []
included_beam_nums = []
included_beam_interleaves = []
included_beam_ids = []
included_beam_sep = []

for tgt in targets:
    target_loc = SkyCoord(ra=tgt['ra_deg_cont']*u.degree, dec=tgt['dec_deg_cont']*u.degree, frame='icrs')
    src_beams, src_beam_sep = get_beams_near_src(target_loc, u_beam_locs, unique_beams)
    for i in range(len(src_beams)):
        comp_names.append(tgt['component_name'])
        comp_ra.append(tgt['ra_deg_cont'])
        comp_dec.append(tgt['dec_deg_cont'])
        included_beam_nums.append(src_beams['beam_id'].data[i])
        included_beam_interleaves.append(src_beams['interleave'].data[i])
        included_beam_ids.append(src_beams['beam_id'].data[i]+src_beams['interleave'].data[i])
        included_beam_sep.append(src_beam_sep.to(u.deg).value[i])

image_params = Table()
image_params['component_name'] = comp_names
image_params['comp_ra'] = comp_ra
image_params['comp_dec'] = comp_dec
image_params['beam_nums'] = included_beam_nums
image_params['beam_interleaves'] = included_beam_interleaves
image_params['beam_ids'] = included_beam_ids
image_params['beam_sep'] = included_beam_sep

image_params_vot = votable.from_table(image_params)
filename = "smc_srcs_image_params.vot"
votable.writeto(image_params_vot, filename)


image_params_vot.get_first_table()

#print (targets['component_name'][i], target_loc)
#print (u_beam_locs[0], len(src_beams))
#print (src_beams['beam_id'], src_beams['interleave'], src_beam_sep)

<Table masked=True length=2029>
component_name  comp_ra   comp_dec  ... beam_ids       beam_sep     
    str14       float64   float64   ...   str3         float64      
-------------- --------- ---------- ... -------- -------------------
J001932-710022  4.887137 -71.006189 ...      00A  0.1441577978901887
J001932-710022  4.887137 -71.006189 ...      00B  0.4619595431400349
J001932-710022  4.887137 -71.006189 ...      00C  0.5226842855746828
J005813-700958 14.555193 -70.166284 ...      24A 0.36965297448806667
J005813-700958 14.555193 -70.166284 ...      25A  0.7492859956132661
J005813-700958 14.555193 -70.166284 ...      30A  0.5331864009359302
J005813-700958 14.555193 -70.166284 ...      18B  0.5472952483701781
J005813-700958 14.555193 -70.166284 ...      24B  0.3698840872228889
J005813-700958 14.555193 -70.166284 ...      25B   0.699974910095779
J005813-700958 14.555193 -70.166284 ...      24C  0.2424437182158453
           ...       ...        ... ...      ...                 ...
J0

In [9]:
ar = np.array(included_beam_ids)
for i in range(36):
    for interleave in ('A', 'B', 'C'):
        key = '{:02d}{}'.format(i, interleave)
        count = len(ar[ar==key])
        print (key, count)
print ('average', len(ar)/(36*3))

00A 21
00B 25
00C 21
01A 8
01B 14
01C 11
02A 10
02B 13
02C 19
03A 25
03B 20
03C 24
04A 18
04B 15
04C 12
05A 4
05B 9
05C 14
06A 21
06B 31
06C 16
07A 17
07B 15
07C 19
08A 20
08B 20
08C 24
09A 16
09B 17
09C 13
10A 11
10B 12
10C 17
11A 17
11B 24
11C 20
12A 30
12B 32
12C 22
13A 19
13B 25
13C 18
14A 24
14B 21
14C 25
15A 22
15B 20
15C 13
16A 8
16B 7
16C 8
17A 13
17B 13
17C 21
18A 26
18B 22
18C 25
19A 24
19B 19
19C 28
20A 28
20B 27
20C 27
21A 20
21B 24
21C 24
22A 11
22B 27
22C 11
23A 15
23B 10
23C 17
24A 23
24B 16
24C 24
25A 24
25B 20
25C 20
26A 23
26B 19
26C 19
27A 20
27B 15
27C 23
28A 24
28B 24
28C 28
29A 16
29B 22
29C 15
30A 24
30B 23
30C 21
31A 17
31B 21
31C 17
32A 17
32B 12
32C 10
33A 10
33B 10
33C 16
34A 22
34B 19
34C 25
35A 18
35B 20
35C 13
average 18.787037037037038


In [10]:
ar = np.array(included_beam_ids)
print (len(ar), len(ar[ar=='02A']))

2029 10


In [11]:

tgt = targets[100]
target_loc = SkyCoord(ra=tgt['ra_deg_cont']*u.degree, dec=tgt['dec_deg_cont']*u.degree, frame='icrs')
src_beams, src_beam_sep = get_beams_near_src(target_loc, u_beam_locs, unique_beams)
print(tgt['component_name'])
print(tgt['ra_deg_cont'])
print(tgt['dec_deg_cont'])
print(src_beams['beam_id'].data)

J002919-722810
7.329157
-72.469711
['01' '02' '07' '08' '01' '02' '01' '02' '07']


## Scheduling of concurrent runs

In [55]:
class CommandFailedError(Exception):
    def __init__(self, value):
        self.value = value

    def __str__(self):
        return repr(self.value)


def run_os_cmd(cmd, failOnErr=True):
    """
    Run an operating system command ensuring that it finishes successfully.
    If the comand fails, the program will exit.
    :param cmd: The command to be run
    :return: None
    """
    print (">", cmd)
    sys.stdout.flush()
    try:
        retcode = subprocess.call(cmd, shell=True)
        if retcode != 0:
            message = "Command '"+cmd+"' failed with code " + str(retcode)
            print (message, file=sys.stderr)
            if failOnErr:
                raise CommandFailedError(message)
    except OSError as e:
        message = "Command '" + cmd + "' failed " + e
        print (message, file=sys.stderr)
        if failOnErr:
            raise CommandFailedError(message)
    return None

In [56]:
# Build map of sources to beam ids
src_beam_map = dict()
for row in image_params:
    comp_name = row['component_name']
    beam_id = row['beam_ids']
    if comp_name not in src_beam_map.keys():
        beams = set()
        src_beam_map[comp_name] = beams
    beams = src_beam_map[comp_name]
    beams.add(beam_id)
#print (src_beam_map)

In [61]:
remaining_array_ids = list(range(1, len(targets)+1))
#remaining_array_ids = list(range(len(targets), 0, -1))

active_ms = set()
active_ids = set()
completed_srcs = set()
#remaining_srcs = set(targets['component_name'])
expected_srcs = len(targets)
status_folder = 'status/8906'

total_concurrency = 0
print ('Processing {} targets (from {})'.format(len(remaining_srcs), expected_srcs))
i =0
while len(remaining_array_ids) > 0 and i < 500:
    i+=1
    print("\nLoop #{}, completed {} remaining {}".format(i, len(completed_srcs), len(remaining_array_ids)))
    # Take a copy of the list to avoid issues when removing items from it
    ids_to_scan = list(remaining_array_ids)
    for array_id in ids_to_scan:
        comp_name = targets['component_name'][array_id-1]
        if comp_name in completed_srcs:
            continue
            
        if os.path.isfile('{}/{:d}.COMPLETED'.format(status_folder, array_id)):
            #print ('--- ' + str(active_ids))
            if array_id in active_ids:
                print('Completed {}  (#{}) concurrency {}'.format(comp_name, array_id, len(active_ids)))
                total_concurrency += len(active_ids)
                active_ids.remove(array_id)
                tgt_ms = src_beam_map[comp_name]
                for ms in tgt_ms:
                    active_ms.remove(ms)
            else:
                print ('Skipping {} (#{}) as it has already completed'.format(comp_name, array_id))
            completed_srcs.add(comp_name)
            remaining_array_ids.remove(array_id)
            continue

        tgt_ms = src_beam_map[comp_name]
        if tgt_ms & active_ms:
            continue
        for ms in tgt_ms:
            active_ms.add(ms)
        active_ids.add(array_id)
        #print ('+++ ' + str(active_ids))
        print('Starting {} (#{}) concurrency {} ms: {}'.format(comp_name, array_id, len(active_ids), tgt_ms))
        #run_os_cmd('./make_askap_abs_cutout.sh {} {}'.format(array_id, status_folder))
        run_os_cmd('./start_job.sh {}'.format(array_id))

    
print ('\nCompleted processing in {} loops with average concurrency {:.2f}'.format(i, total_concurrency/len(targets)))

Processing 354 targets (from 354)

Loop #1, completed 0 remaining 354
Skipping J001932-710022 (#1) as it has already completed
Skipping J005813-700958 (#2) as it has already completed
Skipping J011134-711413 (#3) as it has already completed
Skipping J002248-734007 (#4) as it has already completed
Skipping J014114-740731 (#5) as it has already completed
Skipping J012430-752241 (#6) as it has already completed
Skipping J010218-754651 (#7) as it has already completed
Skipping J013742-733049 (#8) as it has already completed
Skipping J013329-730304 (#9) as it has already completed
Skipping J005221-752539 (#10) as it has already completed
Skipping J005223-752547 (#11) as it has already completed
Skipping J011615-733851 (#12) as it has already completed
Skipping J011610-733900 (#13) as it has already completed
Skipping J003511-710955 (#14) as it has already completed
Skipping J003158-703514 (#15) as it has already completed
Skipping J011131-754505 (#16) as it has already completed
Skipping J0

## Older iterations of processing code

In [24]:
active_ms = set()
active_srcs = deque()
completed_srcs = set()
remaining_srcs = set(targets['component_name'])
expected_srcs = len(targets)

total_concurrency = 0
print ('Processing {} targets (from {})'.format(len(remaining_srcs), expected_srcs))
i =0
while len(remaining_srcs) > 0 and i < 500:
    i+=1
    print("Loop #{}, completed {} remaining {}".format(i, len(completed_srcs), len(remaining_srcs)))
    for comp_name in remaining_srcs:
        #comp_name = targets['component_name'][array_id-1]
        if comp_name in completed_srcs or comp_name in active_srcs:
            continue
        tgt_ms = src_beam_map[comp_name]
        if tgt_ms & active_ms:
            continue
        for ms in tgt_ms:
            active_ms.add(ms)
        active_srcs.append(comp_name)
        print('* Starting {} concurrency {} ms: {}'.format(comp_name, len(active_srcs), tgt_ms))
    
    # Simulate one entry finishing per cycle
    finished_src = active_srcs.popleft()
    tgt_ms = src_beam_map[finished_src]
    completed_srcs.add(finished_src)
    remaining_srcs.remove(finished_src)
    for ms in tgt_ms:
        active_ms.remove(ms)
    print('* Completed {} concurrency {}'.format(finished_src, len(active_srcs)))
    total_concurrency += len(active_srcs)

print ('Completed processing in {} loops with average concurrency {:.2f}'.format(i, total_concurrency/i))

Processing 354 targets (from 354)
Loop #1, completed 0 remaining 354
* Starting J002248-734007 concurrency 1 ms: {'02C', '03A'}
* Starting J004954-715150 concurrency 2 ms: {'19A', '19C', '13B', '19B', '14B', '14C', '14A', '13C', '20A'}
* Starting J010811-754156 concurrency 3 ms: {'11C', '11B', '17C'}
* Starting J012547-731600 concurrency 4 ms: {'33C', '29B', '34A', '28C', '34B', '34C', '28B'}
* Starting J002841-704515 concurrency 5 ms: {'00B', '06B', '06A', '12A', '00C'}
* Starting J011623-703000 concurrency 6 ms: {'30C', '31B', '30B', '31C', '31A'}
* Starting J011035-722807 concurrency 7 ms: {'26C', '21B', '20C', '32A', '20B', '27B', '27A', '33A', '27C'}
* Starting J005636-740315 concurrency 8 ms: {'16B', '09C', '15C', '22A', '17B', '09B', '16C', '10B', '16A'}
* Starting J014810-744345 concurrency 9 ms: {'35C'}
* Starting J003131-743031 concurrency 10 ms: {'03C', '04C', '04B', '04A'}
* Starting J012816-751258 concurrency 11 ms: {'23C'}
* Starting J003917-694842 concurrency 12 ms: {'12